In [1]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import plotly.graph_objects as go
import sys
sys.path.append('..')

from oviz import Trace, TraceCollection, Animate3D

In [2]:
delta_t = -1
static_traces = []
static_traces_times = []


# read in Gordian scatter map
#dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
dfv = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/mustache_work/figures/cubes/edenhofer_scatterized_4pc.csv')
dfv = dfv.loc[dfv['e'].between(0.003, 1)]
#dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 1
scatter_edenhofer = go.Scatter3d(
    x=dfv['x'].values[::ds_index],
    y=dfv['y'].values[::ds_index],
    z=dfv['z'].values[::ds_index],
    mode='markers',
    marker=dict(size=2,
                color='gray',
                symbol='circle',
                opacity=.08),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0

# read in leike map

hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

In [3]:
# Annotations
text_trace = go.Scatter3d(
    x=[100],
    y=[-300],
    z=[-40],
    mode='text',
    text=['Vela OB2 forms'],
    textposition='top center',
    textfont=dict(
        family="Arial", 
        size=12, 
        color="white"
        ),
    name='text',
    hoverinfo='text',
    visible = True,
    showlegend=False
)
# static_traces.append(text_trace)
# static_traces_times.append(np.arange(-5, -15, delta_t)) # only show at t=0

In [4]:
labels = pd.read_csv('/Users/cam/Downloads/sco_cen_grouped_clusters.csv')
df_sc = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/cluster_data/files_formatted/ratzi_2022.csv')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [0], 'n_stars' : [1]})

In [5]:
lcc = ['sig Cen', 'Acrux', 'Musca-foreground', 'eps Cham', 'eta Cham']
df_lcc = df_sc.loc[df_sc['name'].isin(lcc)]

In [6]:
df = pd.read_csv('/Users/cam/Downloads/taurus_core_sigma_final_age.csv')
a = df[['X', 'Y', 'Z', 'U', 'V', 'W', 'labels', 'age_myr']]
a = a.rename(columns={'X' : 'x', 'Y' : 'y', 'Z' : 'z', 'labels' : 'name'})
# for k in df.keys():
#     print(k)

In [7]:
# spica
N = 50

ra = [201.2983] * N
dec = [-11.1612] * N

pmra = -41.58
pmdec = -30.35
pmra_err = 0.2
pmdec_err = 0.19
rv = -0.5
rv_err = 1.1

dist = 80
dist_err = 5

pmra_samples = np.random.uniform(pmra - pmra_err, pmra + pmra_err, N)
pmdec_samples = np.random.uniform(pmdec - pmdec_err, pmdec + pmdec_err, N)
distance_samples = np.random.uniform(dist - dist_err, dist + dist_err, N)
rv_samples = np.random.uniform(rv - rv_err, rv + rv_err, N)
names = ['Spica (RV = {}, pmra = {}, pmdec = {}, distance = {})'.format(rv, pmra, pmdec, dist)] * N

i = SkyCoord(
    ra=ra * u.deg,
    dec=dec * u.deg,
    distance=distance_samples * u.pc,
    pm_ra_cosdec=pmra_samples * (u.mas / u.yr),
    pm_dec=pmdec_samples * (u.mas / u.yr),
    radial_velocity=rv_samples * u.km / u.s,
    frame='icrs'
)
g = i.transform_to('galactic')
g.representation_type = 'cartesian'
g.differential_type = 'cartesian'
spica = pd.DataFrame({
    'x': g.u.value,
    'y': g.v.value,
    'z': g.w.value,
    'U': g.U.value,
    'V': g.V.value,
    'W': g.W.value,
    'name': names,
    'age_myr': [12] * N
})


In [8]:
taurus = a.groupby('name').median().reset_index()
taurus['U_err'] = a.groupby('name')['U'].std() / np.sqrt(a.groupby('name')['U'].count())
taurus['V_err'] = a.groupby('name')['V'].std() / np.sqrt(a.groupby('name')['V'].count())
taurus['W_err'] = a.groupby('name')['W'].std() / np.sqrt(a.groupby('name')['W'].count())
taurus = taurus.loc[taurus['age_myr'] < 5]

In [9]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})
ap = df_hunt_gagne.loc[df_hunt_gagne['family'] == 'alphaPer']
#ap = ap.loc[ap['age_myr'] < 15]
m6 = df_hunt_gagne.loc[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne.loc[df_hunt_gagne['family'] == 'cr135']

young_clusters = df_hunt_gagne.loc[df_hunt_gagne['age_myr'] < 15]
older_clusters = df_hunt_gagne.loc[df_hunt_gagne['age_myr'] > 15]

In [10]:
taurus_data = Trace(taurus, data_name = 'Taurus', min_size = 0, max_size = 12, color = 'white', opacity = 1., marker_style = 'circle', show_tracks=True)
young_clusters_data = Trace(young_clusters, data_name = 'Young clusters (Hunt)', min_size = 0, max_size = 12, color = 'white', opacity = .5, marker_style = 'circle', show_tracks=True)
older_clusters_data = Trace(older_clusters, data_name = 'Older clusters (Hunt)', min_size = 0, max_size = 12, color = 'gray', opacity = .5, marker_style = 'circle', show_tracks=True, size_by_n_stars=True)
full_data = Trace(df_hunt_gagne, data_name = 'All clusters (Hunt)', min_size = 0, max_size = 15, color = 'grey', opacity = .1, marker_style = 'circle')
spica_data = Trace(spica, data_name = 'Spicae', min_size = 5, max_size = 5, color = 'lime', opacity = 1, marker_style = 'circle', show_tracks=True)
ap_data = Trace(ap, data_name = '⍺Per family', min_size = 0, max_size = 13, color = 'violet', opacity = .3, marker_style = 'circle', show_tracks=True)
cr135_data = Trace(cr135, data_name = 'Cr 135', min_size = 0, max_size = 13, color = 'orange', opacity = .3, marker_style = 'circle', show_tracks=True)
sco_cen_data = Trace(df_sc, data_name = 'Sco-Cen (SigMA)', min_size = 0, max_size = 13, color = 'violet', opacity = .5, marker_style = 'circle', show_tracks=True)
lcc_data = Trace(df_lcc, data_name = 'LCC Chain', min_size = 0, max_size = 13, color = 'violet', opacity = .85, marker_style = 'circle', show_tracks=True)
sun_data = Trace(sun, data_name = 'Sun', min_size = 3, max_size = 3, color = 'yellow', opacity = 1, marker_style = 'circle')

#sc = TraceCollection([young_clusters_data, ap_data, cr135_data, sco_cen_data, lcc_data, sun_data, taurus_data])
sc = TraceCollection([young_clusters_data, older_clusters_data, sco_cen_data, sun_data, taurus_data])

plot_3d = Animate3D(
    data_collection = sc, 
    xyz_widths = (1001, 1001, 300), 
    figure_theme = 'dark', 
    trace_grouping_dict=None)

#save_name = '/Users/cam/Downloads/spica.html'
save_name = '/Users/cam/Desktop/astro_research/radcliffe/cam_website_clone/cam_website/interactive_figures/local_sf.html'
#time_int = np.round(np.arange(0., -40 + delta_t, delta_t), 1)
#time_int = np.round(np.arange(0., 20 - delta_t, -delta_t), 1)
time_int = np.round(np.arange(-20, 20 - delta_t, - delta_t), 1)

fig3d = plot_3d.make_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = static_traces, 
    static_traces_times = static_traces_times, 
    static_traces_legendonly=True,
    focus_group = None,
    fade_in_time = 0, # Myr,
    fade_in_and_out = False,
    show_gc_line = False
)